## SegVol inference

In [1]:
# import sys
# sys.path.append('..')
# import importlib
# import adapt_med_seg.pipelines.evaluate as ev
# importlib.reload(ev)
# EvaluateArgs = ev.EvaluateArgs
# EvaluatePipeline = ev.EvaluatePipeline

/Users/hamarmiklos/anaconda3/envs/SegVol/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from adapt_med_seg.pipelines.evaluate import EvaluateArgs, EvaluatePipeline

In [2]:

model_names = ["segvol_baseline"]
dataset_paths = ["../datasets/chaos001"] # assuming notebooks/preprocess_data.ipynb has been run 
modality_lists = [["CT"], ["MRI"]]

for model_name in model_names:
    for modalities in modality_lists:
        for dataset_path in dataset_paths:
            eval_pipeline = EvaluatePipeline(
                evaluate_args=EvaluateArgs(
                    dataset_path=dataset_path,
                    model_name=model_name,
                    cls_idx=0,
                    device='cpu',
                    modalities=modalities, # we look at each modality separately
                    # use_wandb=True,
                )
            )

            results = eval_pipeline.run()
            print(f'Results for {model_name} and modality {modalities[0]} on dataset {dataset_path}:\n{results}')

/Users/hamarmiklos/anaconda3/envs/SegVol/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.vit ViT.__init__:pos_embed: Argument `pos_embed` has been deprecated since version 1.2. It will be removed in version 1.4. please use `proj_type` instead.
  warn_deprecated(argname, msg, warning_category)
/Users/hamarmiklos/anaconda3/envs/SegVol/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)
/Users/hamarmiklos/anaconda3/envs/SegVol/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force

Results for segvol_baseline and modality CT on dataset ../datasets/chaos001:
{'dice': metatensor(0.9680)}


INFO:adapt_med_seg.pipelines.evaluate:Evaluating segvol_baseline on dataset chaos001
Evaluating CHAOS: 100%|██████████| 24/24 [07:45<00:00, 19.41s/batch]

Results for segvol_baseline and modality MRI on dataset ../datasets/chaos001:
{'dice': metatensor(0.6385)}


In [1]:
from adapt_med_seg.data.dataset import MedSegDataset
from adapt_med_seg.pipelines.train import SegVolLightning, TrainingArgs

from pytorch_lightning import Trainer, seed_everything

seed_everything(42)

model_names = ["segvol_baseline"]
dataset_paths = ["../datasets/processed/T2-weighted-MRI_mr_42"] 

for model_name in model_names:
    for dataset_path in dataset_paths:
        training_args = TrainingArgs(
            dataset_path=dataset_path,
            model_name=model_name,
            modalities=["MRI"],
            cls_idx=0,
            device="cpu",
            test_mode=False
            # use_wandb=True,
        )

        train_pipeline = SegVolLightning(training_args)

        _dataset = MedSegDataset(
            processor=train_pipeline._model.processor,
            dataset_path=dataset_path,
            modalities=["MRI"],
            train=True,
        )

        train_pipeline.set_dataset(_dataset)
        train_dataloader, val_dataloader = _dataset.get_train_val_dataloaders()

        trainer = Trainer(
            max_epochs=10,
            accelerator=training_args.device,
            deterministic=True,
            num_sanity_val_steps=1,
        )
        trainer.fit(train_pipeline, train_dataloader, val_dataloader)
        # results = train_pipeline.run()

        # print(f"Results for {model_name} on dataset {dataset_number}:\n{results}")


/Users/serghei/code/uva-ai-msc/dl2/adapt_med_seg/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Seed set to 42
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/serghei/code/uva-ai-msc/dl2/adapt_med_seg/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/serghei/code/uva-ai-msc/dl2/adapt_med_seg/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. Fo

Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/serghei/code/uva-ai-msc/dl2/adapt_med_seg/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


AttributeError: 'list' object has no attribute 'dtype'